In [1]:
import pandas as pd
import numpy as np
from transformers import BartForConditionalGeneration, BartTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

/workspaces/Intelligent-Customer-Feedback-Analysis-System-using-AI/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('dataset/csv/processed_data.csv')
long_feedback = df[df['processed_text'].str.split().str.len() > 50].copy()
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [3]:
def bart_summarize(text, max_length=150, min_length=50):
    inputs = bart_tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = bart_model.generate(
        inputs['input_ids'],
        num_beams=4,
        max_length=max_length,
        min_length=min_length,
        early_stopping=True
    )
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [4]:
sample_texts = long_feedback['processed_text'].head(5).tolist()
bart_summaries = []

In [5]:
for idx, text in enumerate(sample_texts, 1):
    print(f"\nProcessing feedback {idx}/5...")
    short_summary = bart_summarize(text, max_length=50, min_length=20)
    detailed_summary = bart_summarize(text, max_length=150, min_length=50)
    bart_summaries.append({
        'feedback_id': idx,
        'original_text': text[:200] + '...',
        'short_summary': short_summary,
        'detailed_summary': detailed_summary
    })
    print(f"Short Summary: {short_summary}")
    print(f"Detailed Summary: {detailed_summary}")

bart_df = pd.DataFrame(bart_summaries)
bart_df.to_csv('dataset/csv/bart_summaries.csv', index=False)


Processing feedback 1/5...
Short Summary: amazing soundtrack favorite music time hand intense sadness prisoner fate mean youve played game hope distant promise girl stole star important inspiration personally throughout teen year higher energy track like chrono cross time scar time dreamwatch chronomantique indefinably
Detailed Summary: amazing soundtrack favorite music time hand intense sadness prisoner fate mean youve played game hope distant promise girl stole star important inspiration personally throughout teen year higher energy track like chrono cross time scar time dreamwatch chronomantique indefinably remeniscent chrono trigger absolutely superb well.

Processing feedback 2/5...
Short Summary: excellent soundtrack truly like soundtrack enjoy video game music played game music enjoy truly relaxing peacefulon disk one favorite scar time life death forest illusion fortress ancient dragon lost fragment drowned valleydisk two draggons galdorb home chronomantique
Detailed Summary: 

In [6]:
def extractive_summarize(text, num_sentences=3):
    sentences = text.split('.')
    sentences = [s.strip() for s in sentences if len(s.strip()) > 10]
    if len(sentences) <= num_sentences:
        return '. '.join(sentences) + '.'
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    top_sentences = ranked_sentences[:num_sentences]
    top_sentences = sorted(top_sentences, key=lambda x: sentences.index(x[1]))
    
    summary = '. '.join([s[1] for s in top_sentences]) + '.'
    return summary

In [7]:
extractive_summaries = []
for idx, text in enumerate(sample_texts, 1):
    print(f"\nProcessing feedback {idx}/5...")
    short_summary = extractive_summarize(text, num_sentences=2)
    detailed_summary = extractive_summarize(text, num_sentences=4)
    
    extractive_summaries.append({
        'feedback_id': idx,
        'original_text': text[:200] + '...',
        'short_summary': short_summary,
        'detailed_summary': detailed_summary
    })
    
    print(f"Short Summary: {short_summary}")
    print(f"Detailed Summary: {detailed_summary}")

extractive_df = pd.DataFrame(extractive_summaries)
extractive_df.to_csv('dataset/csv/extractive_summaries.csv', index=False)


Processing feedback 1/5...
Short Summary: amazing soundtrack favorite music time hand intense sadness prisoner fate mean youve played game hope distant promise girl stole star important inspiration personally throughout teen year higher energy track like chrono cross time scar time dreamwatch chronomantique indefinably remeniscent chrono trigger absolutely superb wellthis soundtrack amazing music probably best composer work havent heard xenogears soundtrack cant say sure even youve never played game would worth twice price buy iti wish could give star.
Detailed Summary: amazing soundtrack favorite music time hand intense sadness prisoner fate mean youve played game hope distant promise girl stole star important inspiration personally throughout teen year higher energy track like chrono cross time scar time dreamwatch chronomantique indefinably remeniscent chrono trigger absolutely superb wellthis soundtrack amazing music probably best composer work havent heard xenogears soundtrack ca

In [8]:
comparison_data = []
for i in range(len(sample_texts)):
    comparison_data.append({
        'Feedback_ID': i+1,
        'Original_Length': len(sample_texts[i].split()),
        'BART_Short': len(bart_summaries[i]['short_summary'].split()),
        'BART_Detailed': len(bart_summaries[i]['detailed_summary'].split()),
        'Extractive_Short': len(extractive_summaries[i]['short_summary'].split()),
        'Extractive_Detailed': len(extractive_summaries[i]['detailed_summary'].split())
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df)
comparison_df.to_csv('dataset/csv/summary_comparison.csv', index=False)

   Feedback_ID  Original_Length  BART_Short  BART_Detailed  Extractive_Short  \
0            1               73          38             44                73   
1            2               72          37             64                72   
2            3               70          38             70                70   
3            4               71          23             56                71   
4            5               60          43             61                60   

   Extractive_Detailed  
0                   73  
1                   72  
2                   70  
3                   71  
4                   60  


In [9]:
example_idx = 0
summary_output = {
    'total_feedback_analyzed': len(long_feedback),
    'summaries_generated': len(sample_texts),
    'methods_used': ['BART (Transformer)', 'TF-IDF + Cosine Similarity (Extractive)'],
    'output_files': ['dataset/csv/bart_summaries.csv', 'dataset/csv/extractive_summaries.csv', 'dataset/csv/summary_comparison.csv']
}
for key, value in summary_output.items():
    print(f"{key}: {value}")

total_feedback_analyzed: 427
summaries_generated: 5
methods_used: ['BART (Transformer)', 'TF-IDF + Cosine Similarity (Extractive)']
output_files: ['dataset/csv/bart_summaries.csv', 'dataset/csv/extractive_summaries.csv', 'dataset/csv/summary_comparison.csv']
